# Brain Tumor Multiclass Classifier (2D MRI)

This notebook presents a deep learning pipeline for classifying 2D brain MRI scans into four tumor categories: **Glioma**, **Meningioma**, **Pituitary**, and **No Tumor**.

The model is built using **PyTorch** and leverages a pre-trained **DenseNet121** backbone with a custom classifier head. Training is performed in two phases: a warm-up stage where only the head is trained, followed by fine-tuning of selected high-level convolutional blocks.

The dataset used was originally published on [Kaggle](#) and is re-hosted on the **Hugging Face Hub** to simplify access and integration.

This project is part of a personal initiative to explore medical imaging with computer vision, with emphasis on **transfer learning**, **training strategy**, and **model generalization**.

> **Disclaimer:** This project is for **educational and research purposes only**. It is *not* intended for medical or clinical use.


## 1. Setup & Dependencies

This section installs and imports the required libraries for data handling, model development, training, and evaluation.


### 1.1 Install Dependencies (Colab Only)

If you're using Google Colab, install the required packages using the command below.


In [ ]:
# Install Hugging Face datasets library (for Colab users)
!pip install -U datasets fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


### 1.2 Import Libraries

We import all necessary libraries including PyTorch, Albumentations, scikit-learn, and Hugging Face datasets.


In [ ]:
# PyTorch core modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset

# Torchvision for model architectures and data utilities
import torchvision.models as models
from torchvision import datasets

# Albumentations for data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Scikit-learn for model evaluation utilities
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Standard libraries
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm
import copy

# Visualization
import matplotlib.pyplot as plt

# Hugging Face datasets
from datasets import load_dataset

## 2. Load and Prepare Raw Dataset

We load a multiclass brain MRI dataset from the Hugging Face Hub using the `load_dataset` function. This dataset contains four tumor categories and was originally sourced from Kaggle. It is automatically cached for reuse.

The loaded data will be split into training and validation sets using a **stratified sampling** strategy to ensure balanced class representation.


### 2.1 Load Dataset from Hugging Face

We load the dataset directly using the `datasets` library. The dataset contains labeled 2D brain MRI scans across four classes: **glioma**, **meningioma**, **pituitary**, and **no tumor**.


In [ ]:
# Load brain tumor dataset from Hugging Face (auto-cached locally)
ds = load_dataset("Cayanaaa/BrainTumorDatasets", name="multiclass")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

multiclass/train-00000-of-00001.parquet:   0%|          | 0.00/130M [00:00<?, ?B/s]

multiclass/test-00000-of-00001.parquet:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5712 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1311 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5712
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1311
    })
})

### 2.2 View Class Label Mapping

This command reveals the label names and their corresponding integer encodings used internally by the dataset.


In [ ]:
# Display class labels and their corresponding integer indices
print(ds['train'].features['label'].names)

['glioma', 'meningioma', 'notumor', 'pituitary']


### 2.3 Extract Images and Labels from Dataset

We extract the raw image and label pairs from the dataset for further processing.


In [ ]:
# Extract image-label pairs from the training split
train_data = ds['train']
images = train_data['image']
labels = train_data['label']

### 2.4 Stratified Train-Validation Split

To ensure balanced class distribution across the training and validation sets, we perform a stratified split. This minimizes the risk of class imbalance during model training.


In [ ]:
# Split data into training and validation sets while preserving class distribution
train_imgs, val_imgs, train_labels, val_labels = train_test_split(images, labels,
                                                                  test_size=0.2,
                                                                  stratify=labels,
                                                                  random_state=42
                                                                  )

## 3. Dataset Preparation

In this section, we prepare the image dataset by applying preprocessing and augmentation techniques, defining a custom PyTorch `Dataset` class, and creating `DataLoaders` for both training and validation phases.


### 3.1 Define Transformation Pipelines

We define image preprocessing and augmentation pipelines using **Albumentations** to improve generalization and performance.

- The **training pipeline** includes resizing, flipping, distortion, noise, and normalization.
- The **validation pipeline** includes only resizing and normalization to ensure consistent evaluation.


In [ ]:
# Define preprocessing & augmentation for training set
train_T = A.Compose([
    A.Resize(224, 224), # Resize to model input size
    A.HorizontalFlip(p=0.5), # Random horizontal flip
    A.VerticalFlip(p=0.5),  # Random vertical flip
    A.RandomBrightnessContrast(p=0.2),  # Slight brightness/contrast variation
    A.GridDistortion(num_steps=5, distort_limit=0.03, p=0.2),  # Grid-based distortion
    A.GaussNoise(p=0.1), # Add Gaussian noise
    A.Normalize(mean=[0.485, 0.456, 0.406], # Normalize using ImageNet stats
                std=[0.229, 0.224, 0.225]),
    ToTensorV2() # Convert to PyTorch tensor
])

# Define preprocessing for validation set (no augmentation)
val_T = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

### 3.2 Define Custom Dataset Class

We define a custom PyTorch `Dataset` class to:

- Apply the appropriate transformations.
- Return each image and its label in tensor format.


In [ ]:
# Custom Dataset class to load image-label pairs and apply transforms
class LoadDataset(Dataset):
  def __init__(self, images, labels, transform=None):
    self.images = images
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img = self.images[idx]
    img = img.convert('RGB') # Ensure image is in RGB format
    img = np.array(img)

    label = self.labels[idx]

    if self.transform:
      img = self.transform(image=img)['image']

    return img, torch.tensor(label, dtype=torch.long)

### 3.3 Create Dataset & DataLoader

We wrap the image-label pairs using our custom `Dataset` class, and prepare `DataLoaders` to efficiently feed data during training and evaluation.


In [ ]:
# Wrap image and label arrays into Dataset objects
train_dataset = LoadDataset(train_imgs, train_labels, train_T)
val_dataset = LoadDataset(val_imgs, val_labels, val_T)

# Create DataLoaders for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # shuffle for training
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False) # no shuffle for validation

## 4. Model, Optimizer, and Training Setup

We adopt a **transfer learning** approach using a pre-trained **DenseNet121** model. To preserve the visual features learned from ImageNet, all convolutional layers are **frozen**, and we train **only the classifier head**. This initial setup focuses on **feature extraction**, before performing full fine-tuning in a later stage.


### 4.1 Load Pre-trained Model

We load **DenseNet121** with ImageNet weights to leverage powerful low-level feature extraction learned from large-scale natural images.


In [ ]:
# Load DenseNet121 model pre-trained on ImageNet
model = models.densenet121(pretrained=True)

# Freeze all layers in the feature extractor to retain pre-trained representations
for param in model.parameters():
  param.requires_grad = False

# Replace the classifier head to match the number of output classes (4)
model.classifier = nn.Linear(model.classifier.in_features, 4)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 111MB/s]


### 4.2 Define Optimizer, Scheduler, and Device

We use the Adam optimizer to update only the classifier head. A learning rate scheduler reduces the learning rate when validation performance plateaus. GPU is used if available.


In [ ]:
# Configure optimizer to update only the classifier head
early_optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-3)

# Set up learning rate scheduler to reduce LR if validation loss stops improving
scheduler_early = ReduceLROnPlateau(early_optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Automatically use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


### 4.3 Define Weighted Loss Function

To address class imbalance in the training data, we compute class weights and apply them to the cross-entropy loss function.


In [ ]:
# Compute class weights to handle imbalance and reduce bias toward frequent classes
class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight = torch.tensor(class_weight, dtype=torch.float32).to(device)

# Move model to the selected device
model.to(device)

# Define weighted cross-entropy loss
crieterion = nn.CrossEntropyLoss(weight=class_weight)

### 4.4 Define Early Stopping

We implement a custom early stopping mechanism to terminate training when the validation loss no longer improves after a specified number of epochs.


In [ ]:
# Custom early stopping class to monitor validation performance
# Stops training if no improvement is observed over 'patience' epochs
class EarlyStopping:
    def __init__(self, monitor='val_loss', mode='min', patience=3, delta=0.0, verbose=True):
         """
        Args:
            monitor (str): Metric to monitor ('val_loss' or 'val_acc')
            mode (str): 'min' → lower is better, 'max' → higher is better
            patience (int): # of epochs with no improvement before stopping
            delta (float): Minimum change to qualify as improvement
            verbose (bool): Print status each epoch if True
        """
        self.monitor = monitor
        self.mode = mode
        self.patience = patience
        self.delta = delta
        self.verbose = verbose

        self.best_score = None
        self.counter = 0
        self.early_stop = False

        # Set comparison function and initial best value
        if self.mode == 'min':
            self.monitor_op = lambda current, best: current < best - self.delta
            self.best_score = np.inf
        elif self.mode == 'max':
            self.monitor_op = lambda current, best: current > best + self.delta
            self.best_score = -np.inf
        else:
            raise ValueError("mode must be 'min' or 'max'")

    def __call__(self, current_score):
        # Initialize best score
        if self.best_score is None:
            self.best_score = current_score
            if self.verbose:
                print(f"[EarlyStopping] Initial best {self.monitor}: {self.best_score:.4f}")
        # Check for improvement
        elif self.monitor_op(current_score, self.best_score):
            self.best_score = current_score
            self.counter = 0
            if self.verbose:
                print(f"[EarlyStopping] Improved {self.monitor}: {self.best_score:.4f}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"[EarlyStopping] No improvement in {self.monitor} for {self.counter}/{self.patience} epochs.")
            # Stop training if performance has not improved for 'patience' epochs
            if self.counter >= self.patience:
                if self.verbose:
                    print(f"[EarlyStopping] Stopping training. Best {self.monitor}: {self.best_score:.4f}")
                self.early_stop = True

In [ ]:
# Create an EarlyStopping instance to monitor validation loss
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=True)

## 5. Train Classifier Head (Warm-up Phase)

In this phase, we only train the classifier head (fully connected layers) while keeping the backbone frozen. This **warm-up strategy** helps the model gradually adapt to the domain-specific brain MRI data without modifying the general features learned from ImageNet.

The goal is to allow the final layers to specialize on our dataset before unfreezing and fine-tuning the entire network.


In [ ]:
# Save initial model weights and set best validation loss to infinity
best_model_wts = copy.deepcopy(model.state_dict())
best_val_loss = np.inf

num_epoch = 100

for epoch in range(num_epoch):
  print("-" * 50)
  print(f"Epoch {epoch+1}/{num_epoch}")
  print("-" * 50)

  # --- training Phase ---
  model.train()
  train_loss, correct, total = 0.0, 0, 0

  for images, labels in tqdm(train_loader, desc="Training"):
    images, labels = images.to(device), labels.to(device)

    early_optimizer.zero_grad()
    outputs = model(images)
    loss = crieterion(outputs, labels)
    loss.backward()
    early_optimizer.step()

    train_loss += loss.item() * images.size(0)
    _, predicted = torch.max(outputs, dim=1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

  avg_train_loss = train_loss / total
  train_acc = correct / total

  # --- Validation Phase ---
  model.eval()
  val_loss, correct, total = 0.0, 0, 0

  with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Validation"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = crieterion(outputs, labels)

        val_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

  avg_val_loss = val_loss / total
  val_acc = correct / total

  print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}")
  print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}")

  # Step the learning rate scheduler and update early stopping
  scheduler_early.step(avg_val_loss)
  early_stopping(avg_val_loss)

  # Save model weights if validation loss improves
  if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    best_model_wts = copy.deepcopy(model.state_dict())
    torch.save(model.state_dict(), 'mct_best_model.pth')
    print(f"[INFO]: Best model updated")

  # Stop training if early stopping is triggered
  if early_stopping.early_stop:
    print(f"[INFO]: Training Stopped by EarlyStopping")
    break

# Load best model weights after training
model.load_state_dict(best_model_wts)
print("[INFO]: Best Model Loaded")



Epoch 1/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.85it/s]


Train Loss: 0.9025 | Train Acc: 0.67
Val Loss: 0.5652 | Val Acc: 0.83
[EarlyStopping] Improved val_loss: 0.5652
[INFO]: Best model updated
Epoch 2/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.85it/s]


Train Loss: 0.5926 | Train Acc: 0.80
Val Loss: 0.4685 | Val Acc: 0.84
[EarlyStopping] Improved val_loss: 0.4685
[INFO]: Best model updated
Epoch 3/100
--------------------------------------------------


...

Train Loss: 0.3173 | Train Acc: 0.88
Val Loss: 0.2862 | Val Acc: 0.90
[EarlyStopping] No improvement in val_loss for 1/3 epochs.
Epoch 25/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.04it/s]


Train Loss: 0.3052 | Train Acc: 0.89
Val Loss: 0.2645 | Val Acc: 0.90
[EarlyStopping] No improvement in val_loss for 2/3 epochs.
Epoch 26/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.87it/s]

Train Loss: 0.3027 | Train Acc: 0.89
Val Loss: 0.2660 | Val Acc: 0.91
[EarlyStopping] No improvement in val_loss for 3/3 epochs.
[EarlyStopping] Stopping training. Best val_loss: 0.2611
[INFO]: Training Stopped by EarlyStopping
[INFO]: Best Model Loaded


## 6. Fine-Tuning Setup

In this phase, we fine-tune the deeper parts of the model to better adapt to the brain tumor classification task. Instead of unfreezing the entire backbone, we selectively unfreeze the final convolutional block and normalization layer to balance adaptability and generalization.

Fine-tuning allows the model to refine high-level features learned from ImageNet in a domain-specific context.



### 6.1 Unfreeze Selected Layers

Here, we unfreeze the `denseblock4` and `norm5` layers of the backbone while keeping all earlier layers frozen. This selective unfreezing helps avoid overfitting and reduces the risk of catastrophic forgetting.



In [ ]:
# Only unfreeze the last DenseBlock and final batch norm layer (norm5)
for name, param in model.named_parameters():
  if 'denseblock4' in name or 'norm5' in name:
    param.requires_grad = True
  else:
    param.requires_grad = False

### 6.2 Fine-Tuning Optimizer & Callbacks

We define a new optimizer and learning rate scheduler for the fine-tuning phase. Only the parameters marked as trainable (i.e., from `denseblock4` and `norm5`) are updated during this phase.

An `EarlyStopping` callback is also set up to prevent overfitting by halting training when the validation loss no longer improves.

> **Note**: We print the active learning rate after optimizer setup to verify that the new learning rate is properly configured.


In [ ]:
# Define optimizer for fine-tuning (only trainable parameters)
ft_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# Print current learning rate (for verification)
current_lr = ft_optimizer.param_groups[0]['lr']
print(f"Active learning rate: {current_lr}")

# Define scheduler for fine-tuning
scheduler_ft = ReduceLROnPlateau(ft_optimizer, mode='min', factor=0.1, patience=2)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=True)


## 7. Fine-Tune Backbone (Training Loop)

In this section, we perform **fine-tuning** by training the previously unfrozen layers (`denseblock4` and `norm5`) along with the classifier head. Unlike the warm-up phase, this step allows the model to adjust higher-level convolutional features to the specific patterns present in brain MRI images.

The training loop here follows the same structure as the warm-up phase (Section 5), with updated optimizer and scheduler settings defined in Section 6.2. We continue to monitor validation loss and apply **early stopping** to prevent overfitting.


In [ ]:
# Save initial model weights and set best validation loss to infinity
best_model_wts = copy.deepcopy(model.state_dict())
best_val_loss = np.inf

num_epoch = 100

for epoch in range(num_epoch):
  print(f"Epoch {epoch+1}/{num_epoch}")
  print("-" * 50)

  # --- training Phase ---
  model.train()
  train_loss, correct, total = 0.0, 0, 0

  for images, labels in tqdm(train_loader, desc="Training"):
    images, labels = images.to(device), labels.to(device)

    ft_optimizer.zero_grad()
    outputs = model(images)
    loss = crieterion(outputs, labels)
    loss.backward()
    ft_optimizer.step()

    train_loss += loss.item() * images.size(0)
    _, predicted = torch.max(outputs, dim=1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

  avg_train_loss = train_loss / total
  train_acc = correct / total

  # --- Validation Phase ---
  model.eval()
  val_loss, correct, total = 0.0, 0, 0

  with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Validation"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = crieterion(outputs, labels)

        val_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

  avg_val_loss = val_loss / total
  val_acc = correct / total

  print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}")
  print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}")

  scheduler_ft.step(avg_val_loss)
  early_stopping(avg_val_loss)

  if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    best_model_wts = copy.deepcopy(model.state_dict())
    torch.save(model.state_dict(), 'mct_best_model.pth')
    print(f"[INFO]: Best model updated")

  if early_stopping.early_stop:
    print(f"[INFO]: Training Stopped by EarlyStopping")
    break

model.load_state_dict(best_model_wts)
print("[INFO]: Best Model Loaded")



Epoch 1/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.00it/s]


Train Loss: 0.3080 | Train Acc: 0.89
Val Loss: 0.2512 | Val Acc: 0.91
[EarlyStopping] Improved val_loss: 0.2512
[INFO]: Best model updated
Epoch 2/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.97it/s]


Train Loss: 0.2912 | Train Acc: 0.90
Val Loss: 0.2425 | Val Acc: 0.91
[EarlyStopping] Improved val_loss: 0.2425
[INFO]: Best model updated
Epoch 3/100
--------------------------------------------------


...

Train Loss: 0.1001 | Train Acc: 0.97
Val Loss: 0.1050 | Val Acc: 0.97
[EarlyStopping] No improvement in val_loss for 1/3 epochs.
Epoch 54/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.97it/s]


Train Loss: 0.0907 | Train Acc: 0.97
Val Loss: 0.1053 | Val Acc: 0.97
[EarlyStopping] No improvement in val_loss for 2/3 epochs.
Epoch 55/100
--------------------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.87it/s]

Train Loss: 0.0903 | Train Acc: 0.97
Val Loss: 0.1047 | Val Acc: 0.97
[EarlyStopping] No improvement in val_loss for 3/3 epochs.
[EarlyStopping] Stopping training. Best val_loss: 0.1046
[INFO]: Training Stopped by EarlyStopping
[INFO]: Best Model Loaded


## 8. Save Final Model

After fine-tuning, the best-performing model (based on validation loss) is saved using `torch.save()`. This ensures that the most generalizable version of the model is preserved for deployment or further evaluation.

For privacy and reproducibility, the model is uploaded to Hugging Face Hub instead of being stored in a local path. The download link or model reference will be provided in the project source files.

**Model location**: Refer to the model card or config file in the `src/` directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save(model.state_dict(), '/path/to/your/drive')

---

## 📌 Conclusion

This notebook demonstrates how a pre-trained DenseNet121 can be adapted for brain tumor classification through transfer learning. The two-phase training strategy (warm-up and fine-tuning) helps the model converge efficiently while preventing overfitting.

The trained model has been saved and can be used for further evaluation or deployment in downstream tasks.

For evaluation metrics, visualizations, and inference examples, please refer to the `eval_classifier.ipynb` notebook in the project repository.
